In [1]:
import polars as pl

test = pl.read_csv("../data/test_with_index.csv")
data = pl.read_csv("../trained_models/e005-6-7-ens/e005-6-7-ens.csv")

In [8]:
test.shape

(11155, 6)

In [2]:
test = test.with_columns(pl.col("index") + 10000)

In [3]:
data_w_pred = pl.concat([test, data], how="horizontal")

In [4]:
# psuedo_test_data = data_w_pred.filter(
#     (pl.col("target") > 0.99) | (pl.col("target") < 0.20)
# ).with_columns(
#     pl.when(pl.col("target") > 0.99).then(1).otherwise(0).alias("target"),
# )

psuedo_test_data = data_w_pred.with_columns(
    pl.when(pl.col("target") >= 0.50).then(1).otherwise(0).alias("target"),
)

In [5]:
psuedo_test_data["target"].value_counts()

target,count
i32,u32
1,8947
0,2208


In [6]:
import numpy as np
from sklearn.model_selection import StratifiedKFold

fold_arr = np.zeros(psuedo_test_data.height)
sgkf = StratifiedKFold(n_splits=3, random_state=42, shuffle=True)

for idx, (_, val_idx) in enumerate(
    sgkf.split(psuedo_test_data, psuedo_test_data["target"])
):
    fold_arr[val_idx] = idx + 3

psuedo_test_data = psuedo_test_data.with_columns(
    pl.Series(fold_arr).cast(pl.Int64).alias("fold")
)

In [7]:
psuedo_test_data.write_csv("../data/psuedo_all_test_data.csv")